In [1]:
import numpy as np
from sympy import *

# Modeling the deformation of an Insect's wing

Background:

-Wing is modeled as a cantilever beam with a joint and a point load at the free end  





Deflection dependent on:

    -length of beam
    -load
    -EI (Young's modulus, wing's second moment of area) (bending stiffness)
    


Calculaing the second moment of area - I:


In [3]:
w_len = 1
height = 1 
weight = 3
load = 2
young = 1 

def calc_Izz(): #Izz = bh^3 / 12 rectangle
        return (w_len * (height)**3) / 12

In [5]:
I = calc_Izz()
print(I)

0.08333333333333333


In [2]:
class BugWing:

    def __init__(self, w_len, height, joint, young, j_young, d_load):
        #To calculate Moment of Inertia (Izz)
        self.w_len = w_len #aka base
        self.height = height

        #If location == joint, use softer E to calculate dflctn. Track 
        self.joint = joint
        self.young = young
        self.j_young = j_young #only used once.

        self.d_load = d_load 
        self.segmnt_lst = [] 

    def calc_Izz(self): #moment of inertia, rectangle is Izz = bh^3 / 12
        return (self.w_len * (self.height)**3) / 12

    def create_segments(self): #divides wing length into list of wingsegment objects of 2mm length. 2mm ARBITRARY
        i, length = 0, self.w_len
        while length >= 2: #in millimeters
            self.segmnt_lst[i] = WingSegment(3, HOW) #weight force = mass*9.8. WHAT UNITS??
            length -= 3
            i += 1
        if length > 0:
            self.segmnt_lst[i] = WingSegment(length, 0)
        return self.segmnt_lst


In [9]:
class WingSegment(BugWing):
    #all segs are v short. choose standardized length. 
        #right side of ea. beam is cantilevered support
        #left side is unsupported. 
        #left side has force of weight of remaining wing length EXCEPT for the last segment
    
    def __init__(self, s_len, weight, angle = 0, dflctn = 0):
        self.dflctn = dflctn
        self.angle = angle #ANGLE CALCULATION UNNECESSARY??
        self.weight = weight
        self.s_len = s_len

        super().__init__()

    
    def calc_dflctn(self): # v(x) =  ∫∫M(x) / E*I
        init_printing(use_unicode=False, wrap_line=False) #what is this
        x = Symbol('x')
        #what is M(x) ?? M(x) = -(L-x)*(W+q(x))/(2*(L-x))
        bend_eqn = -(self.s_len - x) * (self.weight + self.d_load / 2*(self.s_len - x))
        dflctn_eqn = integrate(integrate(bend_eqn)) / (self.young * self.calc_Izz())
        return dflctn_eqn.evalf(subs={x: self.s_len})
    
    #Updates the dflctn and force angle for each segment in segmnt_lst
    def apply(self): 
        i, lst = 0, BugWing.segmnt_lst
        while lst[i+1]: 
            lst[i].dflctn = self.calc_dflctn()
            #ANGLE CALCULATION UNNECESSARY?
            lst[i+1].angle = 2*asin((0.5 * lst[i].dflctn) / lst[i].length) #assuming lengths are same. HOW FOR LAST SEGMENT? 
            i += 1
        lst[i].dflctn = self.calc_dflctn() #run for last segment

        return BugWing.segmnt_lst

In [10]:
# Constant variables:
E = 1

wing1 = BugWing(1, 1, 1, 1, 1, 1)
wing1.calc_dflct n


In [11]:
init_printing(use_unicode=False, wrap_line=False) #what is this
        x = Symbol('x')
        #what is M(x) ?? M(x) = -(L-x)*(W+q(x))/(2*(L-x))
        bend_eqn = -(self.s_len - x) * (self.weight + self.d_load / 2*(self.s_len - x))
        dflctn_eqn = integrate(integrate(bend_eqn)) / (self.young * self.calc_Izz())
        return dflctn_eqn.evalf(subs={x: self.s_len})

IndentationError: unexpected indent (3660591683.py, line 2)

In [15]:
w_len = 1
height = 1 
weight = 3
load = 2
young = 1 

def calc_Izz(): #moment of inertia, rectangle is Izz = bh^3 / 12
        return (w_len * (height)**3) / 12


In [25]:
def calc_dflctn(): # v(x) =  ∫∫M(x) / E*I
#     init_printing(use_unicode=False, wrap_line=False) #what is this
    x = Symbol('x')
    #what is M(x) ?? M(x) = -(L-x)*(W+q(x))/(2*(L-x))
    bend_eqn = -(w_len - x) * (weight + load / 2*(w_len - x))
    dflctn_eqn = integrate(integrate(bend_eqn)) / (young * calc_Izz())
    return dflctn_eqn.evalf(subs={x: w_len})

In [26]:
calc_dflctn()

-15.0000000000000